In [2]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter

In [3]:
df = pd.read_parquet('users_item_features.parquet')
df.head()

,user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,...,305920,306040,414390,307130,209120,220090,262850,431510,total_playtime_forever,total_playtime_2weeks
0,76561198090457828,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,53.000000,0.000000
1,76561198091690336,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,122.566667,0.000000
2,JakDaRippa,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,399.316667,13.183333
3,Iwannahoneyboobootorapeme,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1120.366667,0.650000
4,76561198003315506,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,113.533333,0.000000


In [4]:
df_matrix = df.iloc[:,1:]
#df_matrix = df.cast(pl.UInt8, strict=False)

In [5]:
users= df.iloc[:,0]

In [6]:
users.head()

0            76561198090457828
1            76561198091690336
2                   JakDaRippa
3    Iwannahoneyboobootorapeme
4            76561198003315506
Name: user_id, dtype: object

In [7]:
users.tail()

73752       capitaomurilao
73753           Mister_Mad
73754        blackbirdflyy
73755    76561198079933051
73756    76561198076301024
Name: user_id, dtype: object

In [8]:
# definimos una función que toma como argumento un id de usuario y devuelve los 5 usuarios más parecidos

def user_user_recomendation(user_id, n_recom=5, umbral=0.95):

    ''' 
    Recomienda usuarios en base a id_de usuario pasado como argumento.
    args:
        user_id:                str, id de usuario al que se va a recomendar.
        n_recom:                int, cantidad de usuarios a recomendar, 5 por defecto.
    returns:
        usuarios_recomendados:  pd.Series, conteniendo los indices e id_usuario de n_recom usuarios.
    '''

    user_id = str(user_id)
    indice= users[users == user_id].index
    similaridades = {}

    if user_id not in users.values:
        return print('Usuario no encontrado. Inserte un id de usuario válido')
    
    try:
        n_recom = int(n_recom)
    except Exception as e:
        print(f'Error {e}')
        print('Cantidad de usuarios a recomendar no válida. Ingrese un entero.')
    if n_recom < 1:
        return print('Inserte una cantidad de usuarios mayor a cero')
    
    contador = 0

    for i in range(len(users)):
        if i != indice:
            sim = cosine_similarity(df_matrix.iloc[indice,:].values.reshape(1,-1), df_matrix.iloc[i,:].values.reshape(1,-1))[0][0]
            similaridades[i] = sim
            if sim > umbral:
                contador +=1
            if contador > n_recom:
                break 
    
    similaridades_sorted = sorted(similaridades.items(), key= itemgetter(1), reverse=True)
    
    #print(f'Similaridades: {similaridades_sorted[0:n_recom]}') ## codigo para verificar

    usuarios_recomendados = []

    for i in range(n_recom):
        usuarios_recomendados.append(similaridades_sorted[i][0])

    return users[usuarios_recomendados]



In [9]:
user_user_recomendation('76561198090457828', n_recom=3, umbral=0.98)

94     76561198087717668
108    76561198044953144
27             yiwpopgug
Name: user_id, dtype: object

In [10]:
pd.set_option('display.max_columns', None)
df[df['user_id'] == '76561198090457828']                # el usuario de prueba

,user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,49520,33440,220200,218740,238090,242130,277430,8870,368230,33930,359320,219640,287980,208090,242700,221100,331470,323370,12210,265210,48700,238460,34900,113200,383080,213670,105600,231180,620,271290,317360,223710,299360,322330,12100,204360,1840,233250,449140,368730,246280,427270,397100,248570,298630,244210,376920,257750,221910,2280,417860,222880,377160,252490,242920,251990,344760,236430,399730,219150,239820,17410,270550,274940,333930,294100,212070,264140,332800,319630,6020,55230,32430,35450,236390,351570,212680,391540,113400,200510,200210,304050,265610,264200,333600,91310,437220,210770,351640,202530,47870,410,253710,339280,203290,233150,346330,295550,250180,227940,17520,113420,320760,8850,233720,219740,220240,333950,388520,107410,206210,299740,209870,322170,362890,413850,346110,311560,48240,266430,316010,249570,240,365590,298050,301520,243870,218230,291550,251570,206420,380770,454890,227680,273350,370480,313740,266210,369200,292030,287700,310950,221380,222420,271590,298110,222440,259530,269210,317470,47410,365450,245620,70,359550,274980,202090,314130,10180,251970,437701,400,6060,31280,8930,237310,230410,231430,225540,7940,234190,33900,261030,243470,302830,4700,203750,355180,275850,400800,433340,242760,352460,322920,234792,297130,397570,278360,441790,265750,233110,322910,276340,448530,304410,274920,219890,223510,268750,360870,12900,335670,215080,244160,270880,8190,247730,233630,303390,208812,243120,234140,204450,225260,50300,201810,212630,427820,500,231040,376300,351040,250600,264710,47790,236110,232790,420,380,339230,349700,226700,209160,219990,21100,215470,401650,290020,248860,244850,298260,370240,233450,214420,365670,221260,369070,419070,24240,39800,113020,238960,310370,260410,383870,204030,207230,9180,10150,214950,261640,34330,259080,256350,249380,337320,265930,10090,218680,251060,22370,307690,248610,250340,221640,104900,9450,262410,222940,221040,286570,263440,108600,208520,239660,495890,472680,295270,283920,294650,432020,12120,241930,428690,227860,228280,240760,250900,9900,236370,381210,205100,346010,239250,313120,17300,206440,209100,209000,218060,224540,242860,9480,359870,22230,203770,200260,19900,241540,17390,253030,295110,274310,252470,344770,22380,202170,245490,319510,20,209080,223470,269250,212500,12160,293260,218130,270210,386070,330830,8980,31270,43000,18700,98800,107100,630,57300,226620,265630,401920,33230,239200,346900,363970,65790,8500,99900,380600,2450,47780,230190,435881,490650,391270,246420,289130,234630,249050,418340,223850,263020,245550,96100,80,56400,257850,239140,322500,385770,252630,212200,387860,387990,310560,253430,203160,213850,369580,282440,8400,225640,421770,239220,236690,17100,223750,65980,400250,296240,320,202970,270370,268650,215530,379720,226860,259700,310380,233270,383980,391460,393010,205890,434260,359290,10500,239160,290300,346560,328220,256410,39120,239070,265770,10,263280,386360,255710,364360,288470,282140,216250,10680,238320,300570,201420,238260,238430,290080,374320,318420,339610,293780,20530,39150,13210,22120,449710,319740,244050,24860,242720,251630,335300,55140,318430,323470,334230,474750,45730,99400,213650,429720,340050,261880,230840,263760,233190,220780,237740,6830,212180,4560,17700,25890,22100,41300,224760,291480,4920,311690,266840,246620,393420,461560,368360,237870,72200,305620,250420,2700,94400,238010,374301,366660,207140,353560,405170,302590,242880,394230,378648,49600,408740,413150,109600,303210,205350,330840,19000,242050,323500,50650,267750,259720,293920,248310,433950,348960,435870,109400,287920,384490,70000,232810,244410,394360,17480,105450,239350,39000,263500,20500,249650,221180,40800,389570,396900,286160,285900,206190,223670,239720,224060,376560,290750,42960,236090,17080,6860,431240,363620,397340,329650,412830,312280,311210,273110,42680,354140,348

In [11]:
df[df['user_id'] == 'Iwannahoneyboobootorapeme']        # un usuario que no recomendó

,user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,49520,33440,220200,218740,238090,242130,277430,8870,368230,33930,359320,219640,287980,208090,242700,221100,331470,323370,12210,265210,48700,238460,34900,113200,383080,213670,105600,231180,620,271290,317360,223710,299360,322330,12100,204360,1840,233250,449140,368730,246280,427270,397100,248570,298630,244210,376920,257750,221910,2280,417860,222880,377160,252490,242920,251990,344760,236430,399730,219150,239820,17410,270550,274940,333930,294100,212070,264140,332800,319630,6020,55230,32430,35450,236390,351570,212680,391540,113400,200510,200210,304050,265610,264200,333600,91310,437220,210770,351640,202530,47870,410,253710,339280,203290,233150,346330,295550,250180,227940,17520,113420,320760,8850,233720,219740,220240,333950,388520,107410,206210,299740,209870,322170,362890,413850,346110,311560,48240,266430,316010,249570,240,365590,298050,301520,243870,218230,291550,251570,206420,380770,454890,227680,273350,370480,313740,266210,369200,292030,287700,310950,221380,222420,271590,298110,222440,259530,269210,317470,47410,365450,245620,70,359550,274980,202090,314130,10180,251970,437701,400,6060,31280,8930,237310,230410,231430,225540,7940,234190,33900,261030,243470,302830,4700,203750,355180,275850,400800,433340,242760,352460,322920,234792,297130,397570,278360,441790,265750,233110,322910,276340,448530,304410,274920,219890,223510,268750,360870,12900,335670,215080,244160,270880,8190,247730,233630,303390,208812,243120,234140,204450,225260,50300,201810,212630,427820,500,231040,376300,351040,250600,264710,47790,236110,232790,420,380,339230,349700,226700,209160,219990,21100,215470,401650,290020,248860,244850,298260,370240,233450,214420,365670,221260,369070,419070,24240,39800,113020,238960,310370,260410,383870,204030,207230,9180,10150,214950,261640,34330,259080,256350,249380,337320,265930,10090,218680,251060,22370,307690,248610,250340,221640,104900,9450,262410,222940,221040,286570,263440,108600,208520,239660,495890,472680,295270,283920,294650,432020,12120,241930,428690,227860,228280,240760,250900,9900,236370,381210,205100,346010,239250,313120,17300,206440,209100,209000,218060,224540,242860,9480,359870,22230,203770,200260,19900,241540,17390,253030,295110,274310,252470,344770,22380,202170,245490,319510,20,209080,223470,269250,212500,12160,293260,218130,270210,386070,330830,8980,31270,43000,18700,98800,107100,630,57300,226620,265630,401920,33230,239200,346900,363970,65790,8500,99900,380600,2450,47780,230190,435881,490650,391270,246420,289130,234630,249050,418340,223850,263020,245550,96100,80,56400,257850,239140,322500,385770,252630,212200,387860,387990,310560,253430,203160,213850,369580,282440,8400,225640,421770,239220,236690,17100,223750,65980,400250,296240,320,202970,270370,268650,215530,379720,226860,259700,310380,233270,383980,391460,393010,205890,434260,359290,10500,239160,290300,346560,328220,256410,39120,239070,265770,10,263280,386360,255710,364360,288470,282140,216250,10680,238320,300570,201420,238260,238430,290080,374320,318420,339610,293780,20530,39150,13210,22120,449710,319740,244050,24860,242720,251630,335300,55140,318430,323470,334230,474750,45730,99400,213650,429720,340050,261880,230840,263760,233190,220780,237740,6830,212180,4560,17700,25890,22100,41300,224760,291480,4920,311690,266840,246620,393420,461560,368360,237870,72200,305620,250420,2700,94400,238010,374301,366660,207140,353560,405170,302590,242880,394230,378648,49600,408740,413150,109600,303210,205350,330840,19000,242050,323500,50650,267750,259720,293920,248310,433950,348960,435870,109400,287920,384490,70000,232810,244410,394360,17480,105450,239350,39000,263500,20500,249650,221180,40800,389570,396900,286160,285900,206190,223670,239720,224060,376560,290750,42960,236090,17080,6860,431240,363620,397340,329650,412830,312280,311210,273110,42680,354140,348

In [13]:
df[df['user_id'] == '76561198044953144']                        # un usuario que sí recomendó

,user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,49520,33440,220200,218740,238090,242130,277430,8870,368230,33930,359320,219640,287980,208090,242700,221100,331470,323370,12210,265210,48700,238460,34900,113200,383080,213670,105600,231180,620,271290,317360,223710,299360,322330,12100,204360,1840,233250,449140,368730,246280,427270,397100,248570,298630,244210,376920,257750,221910,2280,417860,222880,377160,252490,242920,251990,344760,236430,399730,219150,239820,17410,270550,274940,333930,294100,212070,264140,332800,319630,6020,55230,32430,35450,236390,351570,212680,391540,113400,200510,200210,304050,265610,264200,333600,91310,437220,210770,351640,202530,47870,410,253710,339280,203290,233150,346330,295550,250180,227940,17520,113420,320760,8850,233720,219740,220240,333950,388520,107410,206210,299740,209870,322170,362890,413850,346110,311560,48240,266430,316010,249570,240,365590,298050,301520,243870,218230,291550,251570,206420,380770,454890,227680,273350,370480,313740,266210,369200,292030,287700,310950,221380,222420,271590,298110,222440,259530,269210,317470,47410,365450,245620,70,359550,274980,202090,314130,10180,251970,437701,400,6060,31280,8930,237310,230410,231430,225540,7940,234190,33900,261030,243470,302830,4700,203750,355180,275850,400800,433340,242760,352460,322920,234792,297130,397570,278360,441790,265750,233110,322910,276340,448530,304410,274920,219890,223510,268750,360870,12900,335670,215080,244160,270880,8190,247730,233630,303390,208812,243120,234140,204450,225260,50300,201810,212630,427820,500,231040,376300,351040,250600,264710,47790,236110,232790,420,380,339230,349700,226700,209160,219990,21100,215470,401650,290020,248860,244850,298260,370240,233450,214420,365670,221260,369070,419070,24240,39800,113020,238960,310370,260410,383870,204030,207230,9180,10150,214950,261640,34330,259080,256350,249380,337320,265930,10090,218680,251060,22370,307690,248610,250340,221640,104900,9450,262410,222940,221040,286570,263440,108600,208520,239660,495890,472680,295270,283920,294650,432020,12120,241930,428690,227860,228280,240760,250900,9900,236370,381210,205100,346010,239250,313120,17300,206440,209100,209000,218060,224540,242860,9480,359870,22230,203770,200260,19900,241540,17390,253030,295110,274310,252470,344770,22380,202170,245490,319510,20,209080,223470,269250,212500,12160,293260,218130,270210,386070,330830,8980,31270,43000,18700,98800,107100,630,57300,226620,265630,401920,33230,239200,346900,363970,65790,8500,99900,380600,2450,47780,230190,435881,490650,391270,246420,289130,234630,249050,418340,223850,263020,245550,96100,80,56400,257850,239140,322500,385770,252630,212200,387860,387990,310560,253430,203160,213850,369580,282440,8400,225640,421770,239220,236690,17100,223750,65980,400250,296240,320,202970,270370,268650,215530,379720,226860,259700,310380,233270,383980,391460,393010,205890,434260,359290,10500,239160,290300,346560,328220,256410,39120,239070,265770,10,263280,386360,255710,364360,288470,282140,216250,10680,238320,300570,201420,238260,238430,290080,374320,318420,339610,293780,20530,39150,13210,22120,449710,319740,244050,24860,242720,251630,335300,55140,318430,323470,334230,474750,45730,99400,213650,429720,340050,261880,230840,263760,233190,220780,237740,6830,212180,4560,17700,25890,22100,41300,224760,291480,4920,311690,266840,246620,393420,461560,368360,237870,72200,305620,250420,2700,94400,238010,374301,366660,207140,353560,405170,302590,242880,394230,378648,49600,408740,413150,109600,303210,205350,330840,19000,242050,323500,50650,267750,259720,293920,248310,433950,348960,435870,109400,287920,384490,70000,232810,244410,394360,17480,105450,239350,39000,263500,20500,249650,221180,40800,389570,396900,286160,285900,206190,223670,239720,224060,376560,290750,42960,236090,17080,6860,431240,363620,397340,329650,412830,312280,311210,273110,42680,354140,348